## Exercise 1

In this exercise we will understand the functioning of TF/IDF ranking. 

Implement the vector space retrieval model, based on the code framework provided below.

For testing we have provided a simple document collection with 5 documents in file bread.txt:

  DocID | Document Text
  ------|------------------
  1     | How to Bake Breads Without Baking Recipes
  2     | Smith Pies: Best Pies in London
  3     | Numerical Recipes: The Art of Scientific Computing
  4     | Breads, Pastries, Pies, and Cakes: Quantity Baking Recipes
  5     | Pastry: A Book of Best French Pastry Recipes

Now, for the query $Q = ``baking''$, find the top ranked documents according to the TF/IDF rank.

For further testing, use the collection __epfldocs.txt__, which contains recent tweets mentioning EPFL.

Compare the results also to the results obtained from the reference implementation using the scikit-learn library.

In [55]:
# Loading of libraries and documents

from nltk.stem import PorterStemmer, WordNetLemmatizer
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import string
from nltk.corpus import stopwords
import math
from collections import Counter
from operator import itemgetter
nltk.download('stopwords')
nltk.download('punkt')

# Tokenize, stem a document
stemmer = PorterStemmer()
def tokenize(text):
    text = "".join([ch for ch in text if ch not in string.punctuation])
    tokens = nltk.word_tokenize(text)
    return " ".join([stemmer.stem(word.lower()) for word in tokens])

# Read a list of documents from a file. Each line in a file is a document
with open("epfldocs.txt") as f:
    content = f.readlines()
original_documents = [x.strip() for x in content] 
documents = [tokenize(d).split() for d in original_documents]

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/giordanocolombi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/giordanocolombi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [56]:
# TF/IDF code

# create the vocabulary
vocabulary = set([item for sublist in documents for item in sublist])
vocabulary = [word for word in vocabulary if word not in stopwords.words('english')]
vocabulary.sort()

In [57]:
# compute IDF, storing idf values in a dictionary
def idf_values(vocabulary, documents):
    idf = {}
    num_documents = len(documents)
    for i, term in enumerate(vocabulary):
        idf[term] = math.log(num_documents / compute_occurencies(term, documents), math.e)
    return idf

def compute_occurencies(term, documents):
    """count occurencies of a term in a list of documents"""
    return sum([True if term in doc else False for doc in documents])

In [65]:
# Function to generate the vector for a document (with normalisation)
def vectorize(document, vocabulary, idf):
    vector = [0]*len(vocabulary)
    counts = Counter(document)
    max_count = counts.most_common(1)[0][1]
    for i,term in enumerate(vocabulary): 
        vector[i] = idf[term] * counts[term] / max_count
    return vector

# Compute IDF values and vectors
idf = idf_values(vocabulary, documents)
document_vectors = [vectorize(s, vocabulary, idf) for s in documents]

# Function to compute cosine similarity
def cosine_similarity(v1,v2):
    sumxx, sumxy, sumyy = 0, 0, 0
    for i in range(len(v1)):
        x = v1[i]; y = v2[i]
        sumxx += x*x
        sumyy += y*y
        sumxy += x*y
    if sumxy == 0:
            result = 0
    else:
            result = sumxy / math.sqrt(sumxx * sumyy)
    return result

# computing the search result (get the topk documents)
def search_vec(query, topk):
    q = query.split()
    q = [stemmer.stem(w) for w in q]
    query_vector = vectorize(q, vocabulary, idf)
    scores = [[cosine_similarity(query_vector, document_vectors[d]), d] for d in range(len(documents))]
    scores.sort(key=lambda x: -x[0])
    top_docs = []
    for i in range(topk):
            top_docs.append(scores[i][1])
    return top_docs

# HINTS
# natural logarithm function
#     math.log(n,math.e)
# Function to count term frequencies in a document
#     Counter(document)
# most common elements for a list
#     counts.most_common(1)

In [59]:
# Reference code using scikit-learn
tf = TfidfVectorizer(analyzer='word', ngram_range=(1,1), min_df = 1, stop_words = 'english')
features = tf.fit_transform(original_documents)
npm_tfidf = features.todense()
new_features = tf.transform(['baking'])

cosine_similarities = linear_kernel(new_features, features).flatten()
related_docs_indices = cosine_similarities.argsort()[::-1]
topk = 5
for i in range(topk):
    print(original_documents[related_docs_indices[i]])

@Repub_News @EPFL_en you are in Russian media =)
Human genomics of infection and immunity, working closely with engineering and clinical colleagues @EPFL @CHUVLausanne. Please get in touch!
L'app pour «liker» en direct les transports publics https://t.co/3HbkTnIE3Q #epfl
Dr. Julien Faget from @epflSV @EPFL_en will talk about flow cytometry unsupervised analyses: from immune signature to functional biology https://t.co/tAQF8VOLQS
The Swiss Team has won the Solar Decathlon competition  via @EPFL_en / Congrats! @Swiss_Living #VDtech https://t.co/dY437yk4su



## Exercise 2: Evaluate retrieval results

In this exercise, we consider the scikit reference code as an “oracle” that supposedly gives the correct result. Your exercise is to compare the above tf-idf retrieval model with this oracle for the following queries "computer science", "IC school", "information systems" on the **epfldocs.txt** collection.

For this exercise, you need to replace **bread.txt** in the first cell with **epfldocs.txt** and rerun all the cells from the begining. 


In [50]:
# Retrieval oracle 
tf = TfidfVectorizer(analyzer='word', ngram_range=(1,1), min_df = 1, stop_words = 'english')
features = tf.fit_transform(original_documents)
npm_tfidf = features.todense()

In [54]:
tf

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words='english', strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [48]:
# Return all document ids that that have cosine similarity with the query larger than a threshold
def search_vec_sklearn(query, features, threshold=0.1):
    new_features = tf.transform([query])
    cosine_similarities = linear_kernel(new_features, features).flatten()
    related_docs_indices, cos_sim_sorted = zip(*sorted(enumerate(cosine_similarities), key=itemgetter(1), 
                                                       reverse=True))
    doc_ids = []
    for i, cos_sim in enumerate(cos_sim_sorted):
        if cos_sim < threshold:
            break
        doc_ids.append(related_docs_indices[i])
    return doc_ids

In [66]:
ret_ids = search_vec('computer science', 10)
for i, v in enumerate(ret_ids):
    print(original_documents[v])

Exciting News: "World University Rankings 2016-2017 by subject: computer science" No1 @ETH &amp; @EPFL on No8. Congrats https://t.co/ARSlXZoShQ
New computer model shows how proteins are controlled "at a distance" https://t.co/zNjK3bZ6mO  via @EPFL_en #VDtech https://t.co/b9TglXO4KD
An interview with Patrick Barth, a new @EPFL professor who combines protein #biophysics with computer modeling https://t.co/iJwBaEbocj
New at @epfl_en Life Sciences @epflSV: "From PhD directly to Independent Group Leader" #ELFIR_EPFL:  Early Independence Research Scholars. See https://t.co/evqyqD7FFl, also for computational biology #compbio https://t.co/e3pDCg6NVb Deadline April 1 2018 at https://t.co/mJqcrfIqkb
Video of Nicola Marzari from @EPFL_en  on Computational Discovery in the 21st Century during #PASC17 now online: https://t.co/tfCkEvYKtq https://t.co/httPdHcK9W
Exposure Science Film Hackathon 2017 applications open! Come join our Scicomm-film-hacking event! #Science #scicomm https://t.co/zwtKPlh6HT


In [49]:
ret_ids = search_vec_sklearn('computer science', features)
for i, v in enumerate(ret_ids):
    print(original_documents[v])

Exciting News: "World University Rankings 2016-2017 by subject: computer science" No1 @ETH &amp; @EPFL on No8. Congrats https://t.co/ARSlXZoShQ
New computer model shows how proteins are controlled "at a distance" https://t.co/zNjK3bZ6mO  via @EPFL_en #VDtech https://t.co/b9TglXO4KD
An interview with Patrick Barth, a new @EPFL professor who combines protein #biophysics with computer modeling https://t.co/iJwBaEbocj
Exposure Science Film Hackathon 2017 applications open! Come join our Scicomm-film-hacking event! #Science #scicomm https://t.co/zwtKPlh6HT
Le mystère Soulages éblouit la science @EPFL  https://t.co/u3uNICyAdi
@cwarwarrior @EPFL_en @EPFL Doing science at @EPFL_en is indeed pretty cool!!! Thank you for visiting!!!
Blue Brain Nexus: an open-source tool for data-driven science https://t.co/m5yTgXf7ym #epfl
Swiss Data Science on Twitter: "Sign up for @EPFL_en #DataJamDays: learn more a… https://t.co/kNVILHWPGb, see more https://t.co/2wg3BbHBNq
The registration for Exposure Scienc

In [37]:
queries = ["computer science", "IC school", "information systems"]

## Exercise 2.1: Compute the precision and recall at k

In [76]:
def compute_recall_at_k(predict, gt, k):
    correct_predict = set(predict[:k]).intersection(set(gt))
    return len(correct_predict) / len(gt)

In [77]:
def compute_precision_at_k(predict, gt, k):
    correct_predict = set(predict[:k]).intersection(set(gt))
    return len(correct_predict) / k

## Exercise 2.2: Compute the MAP score at 10

In [82]:
def compute_interpolated_precisions(prec_rec):
    max_prec = []
    current_rec = prec_rec[-1][1]
    for k, (prec, rec) in enumerate(prec_rec):
        max_p = prec
        for j, (p, r) in enumerate(prec_rec[k+1:]):
            if p > max_p and r >= rec:
                max_p = p
        max_prec.append(max_p)
    return max_prec

In [85]:
def compute_map(queries):
    map_score = 0
    prec_rec_dict = []
    for i, query in enumerate(queries):
        predict = search_vec(query, 10)
        gt = search_vec_sklearn(query, features)
        prec_rec = []
        for k in range(1, len(gt)+1):
            precision_at_k = compute_precision_at_k(predict, gt, k)
            recall_at_k = compute_recall_at_k(predict, gt, k)
            prec_rec.append((precision_at_k, recall_at_k))
        precs_int = compute_interpolated_precisions(prec_rec)
        print(precs_int)
        map_score += sum(precs_int)/len(gt)
        prec_rec_dict.append(prec_rec)
    map_score = map_score/len(queries)
    return map_score, prec_rec_dict

In [86]:
compute_map(queries)

[1.0, 1.0, 1.0, 0.75, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.6363636363636364, 0.5833333333333334, 0.5384615384615384, 0.5, 0.4666666666666667, 0.4375, 0.4117647058823529, 0.3888888888888889, 0.3684210526315789, 0.35, 0.3333333333333333, 0.3181818181818182, 0.30434782608695654, 0.2916666666666667, 0.28, 0.2692307692307692]
[1.0, 1.0, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.8181818181818182, 0.75, 0.6923076923076923, 0.6428571428571429, 0.6, 0.5625, 0.5294117647058824]
[1.0, 0.6, 0.6, 0.6, 0.6, 0.5, 0.5, 0.5, 0.5, 0.5]


(0.6521383430442943,
 [[(1.0, 0.038461538461538464),
   (1.0, 0.07692307692307693),
   (1.0, 0.11538461538461539),
   (0.75, 0.11538461538461539),
   (0.6, 0.11538461538461539),
   (0.6666666666666666, 0.15384615384615385),
   (0.5714285714285714, 0.15384615384615385),
   (0.625, 0.19230769230769232),
   (0.6666666666666666, 0.23076923076923078),
   (0.7, 0.2692307692307692),
   (0.6363636363636364, 0.2692307692307692),
   (0.5833333333333334, 0.2692307692307692),
   (0.5384615384615384, 0.2692307692307692),
   (0.5, 0.2692307692307692),
   (0.4666666666666667, 0.2692307692307692),
   (0.4375, 0.2692307692307692),
   (0.4117647058823529, 0.2692307692307692),
   (0.3888888888888889, 0.2692307692307692),
   (0.3684210526315789, 0.2692307692307692),
   (0.35, 0.2692307692307692),
   (0.3333333333333333, 0.2692307692307692),
   (0.3181818181818182, 0.2692307692307692),
   (0.30434782608695654, 0.2692307692307692),
   (0.2916666666666667, 0.2692307692307692),
   (0.28, 0.2692307692307692),
